In [1]:
import torch.nn as nn 
import torch 

class Policy(nn.Module):
    def __init__(self):
        super().__init__()
        self.core = nn.Sequential(
            nn.Linear(4,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,2)
        )
        
    def forward(self,x):
        return self.core(x)

class Value(nn.Module):
    def __init__(self):
        super().__init__()
        self.core = nn.Sequential(
            nn.Linear(4,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,1)
        )
        
    def forward(self,x):
        return self.core(x)

policyModel = Policy()
valueModel = Value()

In [2]:
import gymnasium as gym 

env = gym.make("CartPole-v1",render_mode='human')

In [ ]:
def is_terminal(observation):
    x,v,theta , omega = observation 
    x_terminal = x <= -4.8 or x>=4.8
    theta_terminal = theta <= -0.209*7.5 or theta >= 0.209*7.5
    return x_terminal or theta_terminal


episodes = 1000
observations = []
rewards = []
actions = []
value_optimizer = torch.optim.Adam(valueModel.parameters(), lr=1e-3)
valueLoss = torch.nn.functional.mse_loss
gamma = 0.99
for episode in range(episodes):
    done = False
    obs, info = env.reset(seed=42)
    obs = torch.tensor(obs)
    while not done:
        observations.append(obs)
        logits = policyModel(obs)
        probs = torch.softmax(logits,dim=-1)
        action = torch.multinomial(probs, num_samples=1).item()
        actions.append(action)
        obs, reward, terminated, truncated , info = env.step(action)
        rewards.append(reward)
        obs = torch.tensor(obs)
        done = is_terminal(obs)
    print('episode: ',episode + 1)
    

    discontinued_rewards = []
    reward = 0 
    observation_tensor = torch.stack(observations) 
    for i in range(-1,-len(rewards),-1):
        reward = rewards[i] + gamma * reward
        discontinued_rewards.append(reward)


    discontinued_rewards.reverse()
    discontinued_rewards = torch.tensor(discontinued_rewards) 
    
    
    reward_preds = valueModel(observation_tensor)
    
    loss = valueLoss(reward_preds,discontinued_rewards)
    value_optimizer.zero_grad()
    loss.backward()
    value_optimizer.step()
    



    

    

c:\Users\beca\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\classic_control\cartpole.py:214: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


episode:  1
torch.Size([36, 1])
tensor(21.7251, grad_fn=<MseLossBackward0>)


C:\Users\beca\AppData\Local\Temp\ipykernel_20620\1344665949.py:46: UserWarning: Using a target size (torch.Size([35])) that is different to the input size (torch.Size([36, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = valueLoss(reward_preds,discontinued_rewards)
